In [ ]:
import os
from chesslm.lichess import LichessPGNReader, LichessPGNEntry

DATA_DIR = os.path.expanduser("~/Data/lichess")
FILENAME, TOTAL_GAMES = (
    "standard_rated_2013-01", 121_332
    # "standard_rated_2017-02", 10_194_939
    # "standard_rated_2019-03", 34_869_171
    # "standard_rated_2019-10", 40_440_254
)

def accept_entry(entry: LichessPGNEntry) -> bool:
    if entry.termination != "Normal":
        return False
    return entry.ends_with_checkmate or entry.result == "1/2-1/2"

reader = LichessPGNReader(
    os.path.join(DATA_DIR, f"lichess_db_{FILENAME}.pgn.zst"),
    TOTAL_GAMES
)

In [ ]:
%%time
import pyarrow as pa

tb = pa.Table.from_pylist(
    [entry.model_dump() for entry in reader.tqdm_filter(accept_entry)]
)

Retrieved 0 entries:   0%|          | 0.00/10.2M [00:00<?, ?it/s]

Retrieved 3,097,243 entries: 100%|██████████| 10.2M/10.2M [02:21<00:00, 72.3kit/s]


CPU times: user 2min 20s, sys: 2.28 s, total: 2min 23s
Wall time: 2min 25s


In [ ]:
%%time
import gzip
import orjson
from tqdm import tqdm

with gzip.open(os.path.join(DATA_DIR, f"{FILENAME}_filtered.jsonl.gz"), 'wb') as f:
    for row in tqdm(tb.to_pylist(), desc="Writing lines to file"):
        f.write(orjson.dumps(row) + b"\n")

CPU times: user 1min 48s, sys: 202 ms, total: 1min 48s
Wall time: 1min 49s
